# QR Code Transaction Creator and Checker

Background


Shops need to have a fast way of accepting digital transactions. As indicated by Bitcoin analyst, Andreas Antonopolous, for small bitcoin transactions, waiting for confirmations is as unneccessary as waiting for a credit card company to comfirm. Hence this notebook will focus on instant creation of a bitcoin transaction qr code and checking for successful broadcasting of the transacttion.



Goals:

- Take input as value of item in rand
- Convert value to BTC using live price from Luno
- Display QR code to customers with custom transaction
- Monitor Blochain.info for broadcast transaction
- Display payment success when transaction found

In [45]:
from requests import *
from webbrowser import open_new_tab
from websocket import create_connection
import json

In [46]:
#Enter Details of Product/total price and some description (possibly just default to shop name)

#Rand Value of Products
randVal = 125.56

#Description of Products
desc = "billys-coffee-shop"

In [47]:
# Hard-code your bitcoin address
my_BTC_address = "1CGVT1BTV6Q2wpSm1R2vp8L4nCPgq4BuCQ"

In [48]:
#Calculate value of item in BTC using Current Luno Price

#Get Luno price
luno_xrate = get("https://api.mybitx.com/api/1/ticker?pair=XBTZAR").json()['ask']

#Calculate the BTC value from the rand value
amount = round(randVal/float(luno_xrate),8)

In [49]:
#Generate QR Code

#Generate the URL for creating the QR Code
btcURL = str("https://chart.googleapis.com/chart?chs=250x250&cht=qr&&chl=bitcoin:" + 
            my_BTC_address + 
            "?amount=" +
            str(amount) +
            "&label=" +
            desc)

In [50]:
#One possibility is to just open each new generated URL in a browser;
#maybe the application will work in the browser eventually
open_new_tab(btcURL)

True

Example Screenshot of Generated QR Code
<img src="http://github.com/tom2718">

In [51]:
#Check status of Transaction

#Subribe to WebSocket feed of blockchain.info for any new transactions sent to your Address
ws = create_connection("wss://ws.blockchain.info/inv")
ws.send(json.dumps({"op":"unconfirmed_sub"}))     #{"op":"addr_sub", "addr":my_BTC_address})) #Uncomment the different "op" for desired check
    
print("pending")
result =  ws.recv()
# print (result)
print("success")
ws.close()

pending
success


In [52]:
#Function to Confirm a Transaction send a specific address a predefined amount within 1 satoshi
def confirm_tx(outs, satValue, addr="1CGVT1BTV6Q2wpSm1R2vp8L4nCPgq4BuCQ"):
    for out in outs:
        if satValue-1<=out["value"]<=satValue+1 and out["addr"]==addr:
            return True
        else:
            print ("out_value:",out["value"],"out_addr:",out["addr"])
            return False

In [53]:
#Confirm Transaction

#Confirm the amount sent to our address
# if randVal <= 10000:
if(confirm_tx(json.loads(result)["x"]["out"], int(amount*1e8))):
    print("Transaction Received")

#Confirm that there is a tx fee greater than a threshold
#Sum the inputs
total_in = 0
for inpt in json.loads(result)["x"]["inputs"]:
#     if len(inpt)>2: #Ignore script only input
    total_in += inpt["prev_out"]["value"]

print("Total Inputs:", total_in)
    
#Sum the Outputs
total_out = 0
for outpt in json.loads(result)["x"]["out"]:
    total_out += outpt["value"]

#Check leftover amount i.e. Tx fee
if total_in - total_out > 5e4: #Minimum transaction fee = 50 000 Satoshi
    print ("Success! Payment Confirmed.")
else:
    print("Transaction fee too low:", total_in-total_out)


out_value: 10600059 out_addr: 15V1aJKy71x7v792KDNLC2aSknqXvWmQkB
Total Inputs: 93464634
Transaction fee too low: 27677


#### Conclusion and To Dos

The entire confirmation of the transaction should be done in a while loop to monitor event as the websocket pushes them to us. Then we deal with each event.